In [15]:
import numpy as np
from model import CNN
from pysr import PySRRegressor
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch

### Load Model and get Kernels

In [16]:
cnn = CNN()
cnn.load_state_dict(torch.load('cnn.pt'))
print(cnn)

for name, param in cnn.named_parameters():
    if name == 'conv1.weight':
        kernels1 = param
        break
print(f"first kernel of first layer:\n{kernels1[0]}")

/tmp/ipykernel_24641/2465468848.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn.load_state_dict(torch.load('cnn.pt'))


CNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (out): Linear(in_features=1568, out_features=10, bias=True)
)
first kernel of first layer:
tensor([[[-0.2335, -0.2440, -0.9097, -1.0703, -0.8168],
         [-0.0917,  0.0114, -0.2991, -0.3230,  0.6103],
         [ 0.0865,  0.0287,  0.0781,  0.3111,  0.9103],
         [ 0.0553, -0.7562, -0.8900, -0.9722, -0.9644],
         [ 0.0107, -0.6425, -0.8105, -0.5706, -1.1133]]],
       grad_fn=<SelectBackward0>)


### Load Dataset and get results

In [17]:
test_data = datasets.MNIST(root='data', train=False, transform=ToTensor(),)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=10, shuffle=True, num_workers=1)
samples, labels = next(iter(test_loader))

cnn.eval()
with torch.no_grad():
    results = cnn(samples)

### Prepare Data for PySR (Kernel 1)
Extract the 5x5 submatrices (incl. padding) from images to use them as input

In [18]:
kernel_size = 5
X = None
for x in samples:
    x = torch.nn.functional.pad(input=x[0], pad=(2, 2, 2, 2), mode="constant", value=0)
    for i, j in np.ndindex((x.size()[0] - kernel_size + 1, x.size()[1] - kernel_size + 1)):
        slice = x[i:i + kernel_size, j:j + kernel_size]
        if X is None:
            X = np.array([slice.numpy().flatten()])
        else:
            X = np.concatenate((X, [slice.numpy().flatten()]))


y = results['relu1'][:, 0].numpy().flatten()

### Symbolic Regression

In [21]:
regr = PySRRegressor(
    niterations=40,
    binary_operators=["+", "*", "-", "/"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "square",
        "cube",
        "inv(x) = 1/x",  # Julia syntax
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},  # Sympy syntax
    elementwise_loss="loss(prediction, target) = (prediction - target)^2",  # Julia syntax
    warm_start=False,
    progress=False,
)

regr.fit(X, y)

/home/croniev/Code/cysec/lib/python3.9/site-packages/pysr/sr.py:2059: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 2.030e+04
Head worker occupation: 10.8%
Progress: 49 / 600 total iterations (8.167%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
2           2.133e-04  7.971e+00  y = square(0.02125)
4           2.123e-04  2.352e-03  y = cube(0.14472) * x₁₄
5           2.123e-04  1.544e-05  y = cube(0.14472) * sin(x₁₄)
6           2.123e-04  1.831e-04  y = square(cube(-0.40797)) * sin(x₁₄)
8           2.120e-04  5.478e-04  y = square(-0.023006 * exp(x₁₄ - square(x₃)))
9           2.116e-04  2.056e-03  y = square(square(square(x₁₃ / square(x₁₈ + 1.2803))))
10          2.116e-04  3.260e-05  y = (x₁₄ * (exp(sin(0.48156)) - x₄)) * cube(0.14472)
16          2.105e-04  8.382e-04  y = cube((x₉ / cube(exp(sin(x₉ + x₂)))) / cos(sin(cube(-1.1298...
                                   - x₂₁))))
----------------------------------------------------------------------------


Expressions evaluated per second: 1.230e+04
Head worker occupation: 8.5%
Progress: 210 / 600 total iterations (35.000%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
2           2.133e-04  7.971e+00  y = square(0.02125)
3           2.131e-04  8.168e-04  y = 0.0006005 * x₉
4           2.123e-04  3.886e-03  y = cube(0.14472) * x₁₄
5           2.123e-04  1.544e-05  y = cube(0.14472) * sin(x₁₄)
6           2.118e-04  2.321e-03  y = (x₁₄ - x₂₃) * cube(0.18627)
7           2.116e-04  8.738e-04  y = cube(square(square(square(x₁₄)) - x₁₄))
8           2.103e-04  6.424e-03  y = sin(((x₁₄ - x₁₈) * -0.0083423) / -1.0453)
10          2.059e-04  1.060e-02  y = square(0.05527 * cube(sin(-0.48805 + x₁₉) - x₁₄))
11          1.985e-04  3.644e-02  y = square(square(sin(x₁₃ / square(x₁₈ + (1.2803 + x₃)))))
13          1.975e-04  2.612e-03  y = square(square(sin(x₁₃ / square((x₁₈ + x₃) + (1.2803 

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                 square(0.02124964)   
	1         0.004674                                 x14 * 0.0037314403   
	2         0.000361                            sin(x14) * 0.0037314403   
	3         0.001112                     (x14 - x23) * cube(0.18379931)   
	4         0.000902          (x14 / (0.11078131 + x7)) * 0.00076745497   
	5         0.006176     sin(((x14 - x18) * -0.008342263) / -1.0453087)   
	6         0.023686  square(-0.050019395 * cube((-0.6835377 + x19) ...   
	7         0.015800  square(sin((x14 * -0.6835377) * cube(-0.683537...   
	8         0.126462  cube(square(x13 / (x8 + cube(x18 + cos(0.53372...   
	9         0.015323  square(square(sin(square(x13 / (cube(x18 + cos...   
	10        0.019028  cube(square(square(sin(sin(x13 / (x8 + cube(x1...   
	11        0.069691  cube(cube(cube(sin(x13 / (x8 + cube(x18 + (squ...   
	12  >>>>  0.571143  cube(cube(cube(cube(sin(x13 / (x8 + cube(x18 +...   
	13        0.006968  square(square(square(square(square(cube(sin(x1...   
	14        0.047575  cube(cube(square(square(cube(cube(sin(x13 / (c...   
	15        0.000103  cube(cube(square(square(cube(cube(sin(x13 / (c...   
	
	        loss  complexity  
	0   0.000213           2  
	1   0.000212           3  
	2   0.000212           4  
	3   0.000212           6  
	4   0.000212           7  
	5   0.000210           8  
	6   0.000205           9  
	7   0.000202          10  
	8   0.000178          11  
	9   0.000173          13  
	10  0.000169          14  
	11  0.000147          16  
	12  0.000083          17  
	13  0.000083          18  
	14  0.000079          19  
	15  0.000079          20  
]


Expressions evaluated per second: 1.380e+04
